In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../database.csv')
data

/tmp/ipykernel_229028/3357949180.py:1: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../database.csv')


,departure,return,departure_id,departure_name,return_id,return_name,distance (m),duration (sec.),avg_speed (km/h),departure_latitude,departure_longitude,return_latitude,return_longitude,Air temperature (degC)
0,2020-03-23 06:09:44,2020-03-23 06:16:26,86,Kuusitie,111.0,Esterinportti,1747.0,401.0,0.261397,60.195245,24.901900,60.197572,24.926781,0.9
1,2020-03-23 06:11:58,2020-03-23 06:26:31,26,Kamppi (M),10.0,Kasarmitori,1447.0,869.0,0.099908,60.168610,24.930537,60.165017,24.949473,0.9
2,2020-03-23 06:16:29,2020-03-23 06:24:23,268,Porolahden koulu,254.0,Agnetankuja,1772.0,469.0,0.226695,60.195540,25.053581,60.187234,25.036412,0.9
3,2020-03-23 06:33:53,2020-03-23 07:14:03,751,Vallipolku,106.0,Korppaanmäentie,7456.0,2406.0,0.185935,60.227827,24.819614,60.203474,24.898930,0.9
4,2020-03-23 06:36:09,2020-03-23 07:04:10,62,Länsisatamankatu,121.0,Vilhonvuorenkatu,7120.0,1679.0,0.254437,60.158928,24.909692,60.186463,24.967872,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12157453,2017-10-30 23:43:00,2017-10-30 23:55:00,64.0,Tyynenmerenkatu,64.0,Tyynenmerenkatu,918.0,714.0,0.077143,60.156838,24.920661,60.156838,24.920661,0.4
12157454,2017-10-30 23:49:00,2017-10-31 04:49:00,117.0,Brahen puistikko,44.0,Sörnäinen (M),822.0,252.0,0.195714,60.189671,24.948102,60.187736,24.960980,0.4
12157455,2017-10-30 23:52:00,2017-10-31 00:02:00,142.0,Koskelantie,141.0,Intiankatu,1817.0,594.0,0.183535,60.208749,24.946514,60.207858,24.967638,0.4
12157456,2017-10-30 23:57:00,2017-10-31 00:00:00,28.0,Lastenlehto,26.0,Kamppi (M),416.0,152.0,0.164211,60.165812,24.927021,60.168610,24.930537,0.4


In [1]:
import boto3

In [53]:
import logging
import boto3
from botocore.exceptions import ClientError
import json

AWS_REGION = 'eu-west-1'

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

endpoint_url = "http://localhost:4566"
sqs_resource = boto3.resource("sqs", region_name=AWS_REGION, endpoint_url=endpoint_url)

In [54]:
def create_queue(queue_name, delay_seconds, visiblity_timeout):
    """
    Create a standard SQS queue
    """
    try:
        response = sqs_resource.create_queue(QueueName=queue_name,
                                             Attributes={
                                                 'DelaySeconds': delay_seconds,
                                                 'VisibilityTimeout': visiblity_timeout
                                             })
    except ClientError:
        logger.exception(f'Could not create SQS queue - {queue_name}.')
        raise
    else:
        return response

In [64]:
sqs_client = boto3.client("sqs", region_name=AWS_REGION, endpoint_url=endpoint_url)

def get_queue(queue_name):
    """
    Returns the URL of an existing Amazon SQS queue.
    """
    try:
        response = sqs_client.get_queue_url(QueueName=queue_name)['QueueUrl']

    except ClientError:
        logger.exception(f'Could not get the {queue_name} queue.')
        raise
    else:
        return response

get_queue('123')

2022-08-19 13:23:35,494: ERROR: Could not get the 123 queue.
Traceback (most recent call last):
  File "/tmp/ipykernel_11456/3324364624.py", line 8, in get_queue
    response = sqs_client.get_queue_url(QueueName=queue_name)['QueueUrl']
  File "/home/ndx/.local/lib/python3.8/site-packages/botocore/client.py", line 508, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/home/ndx/.local/lib/python3.8/site-packages/botocore/client.py", line 915, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.QueueDoesNotExist: An error occurred (AWS.SimpleQueueService.NonExistentQueue) when calling the GetQueueUrl operation: The specified queue does not exist for this wsdl version.


QueueDoesNotExist: An error occurred (AWS.SimpleQueueService.NonExistentQueue) when calling the GetQueueUrl operation: The specified queue does not exist for this wsdl version.

In [56]:
def delete_queue(queue_name):
    """
    Deletes the queue specified by the QueueUrl.
    """
    try:
        response = sqs_client.delete_queue(QueueUrl=queue_name)

    except ClientError:
        logger.exception(f'Could not delete the {queue_name} queue.')
        raise
    else:
        return response

In [57]:
def send_queue_message(queue_url, msg_attributes, msg_body):
    """
    Sends a message to the specified queue.
    """
    try:
        response = sqs_client.send_message(QueueUrl=queue_url,
                                           MessageAttributes=msg_attributes,
                                           MessageBody=msg_body)
    except ClientError:
        logger.exception(f'Could not send meessage to the - {queue_url}.')
        raise
    else:
        return response

In [58]:
def receive_queue_message(queue_url):
    """
    Retrieves one or more messages (up to 10), from the specified queue.
    """
    try:
        response = sqs_client.receive_message(QueueUrl=queue_url)
    except ClientError:
        logger.exception(
            f'Could not receive the message from the - {queue_url}.')
        raise
    else:
        return response


def delete_queue_message(queue_url, receipt_handle):
    """
    Deletes the specified message from the specified queue.
    """
    try:
        response = sqs_client.delete_message(QueueUrl=queue_url,
                                             ReceiptHandle=receipt_handle)
    except ClientError:
        logger.exception(
            f'Could not delete the meessage from the - {queue_url}.')
        raise
    else:
        return response

In [59]:

create_queue('test-queue', '0', '30')

sqs.Queue(url='http://localhost:4566/000000000000/test-queue')

In [60]:
get_queue('test-queue')

'http://localhost:4566/000000000000/test-queue'

In [61]:
send_queue_message(get_queue('test-queue'), {}, 'test-message')

{'MD5OfMessageBody': 'beaa0032306f083e847cbf86a09ba9b2',
 'MessageId': 'a393f328-43cb-46fb-b463-7ab5e9673658',
 'ResponseMetadata': {'RequestId': 'XBCOK33ACADNTKZ85GJ0SP4Y248EIADG9P781SFTYIHQXYVE2X6T',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'content-length': '412',
   'connection': 'close',
   'access-control-allow-origin': '*',
   'access-control-allow-methods': 'HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH',
   'access-control-allow-headers': 'authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-amz-date,x-amz-request-id,x-amz-security-token,x-amz-tagging,x-amz-target,x-amz-user-agent,x-amz-version-id,x-amzn-requestid,x-localstack-target,amz-sdk-invocation-id,amz-sdk-request',
   'access-control-expose-headers': 'etag,x-amz-version-id',
   'date': 'Fri, 19 Aug 2022 10:08:55 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [62]:
QUEUE_URL = get_queue('test-queue')

messages = receive_queue_message(QUEUE_URL)

print(messages)
for msg in messages['Messages']:
    msg_body = msg['Body']
    receipt_handle = msg['ReceiptHandle']

    logger.info(f'The message body: {msg_body}')
    logger.info('Deleting message from the queue...')

    delete_queue_message(QUEUE_URL, receipt_handle)

2022-08-19 13:08:58,727: INFO: The message body: test-message
2022-08-19 13:08:58,728: INFO: Deleting message from the queue...


{'Messages': [{'MessageId': 'a393f328-43cb-46fb-b463-7ab5e9673658', 'ReceiptHandle': 'MDM5NGExMjAtNDA2OS00MTMxLTkwYmYtMjEzOTNjOWNmNjI5IGFybjphd3M6c3FzOmV1LXdlc3QtMTowMDAwMDAwMDAwMDA6dGVzdC1xdWV1ZSBhMzkzZjMyOC00M2NiLTQ2ZmItYjQ2My03YWI1ZTk2NzM2NTggMTY2MDkwMzczOC43MjUxODk0', 'MD5OfBody': 'beaa0032306f083e847cbf86a09ba9b2', 'Body': 'test-message'}], 'ResponseMetadata': {'RequestId': 'AI0N5GU8YTS9F3LR6B61FJR3MYA3QYJBCFTE014G0534IWGL6OFL', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml', 'content-length': '669', 'connection': 'close', 'access-control-allow-origin': '*', 'access-control-allow-methods': 'HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH', 'access-control-allow-headers': 'authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-amz-date,x-amz-request-id,x-amz-security-token,x-amz-tagging,x-amz-target,x-amz-user-agent,x-amz-version-id,x-amzn-requestid,x-localstack-target,amz-sdk-invocation-id,amz-sdk-request', 'acc

In [5]:
import boto3

AWS_REGION = "eu-west-1"

client = boto3.client("s3", region_name=AWS_REGION, endpoint_url=endpoint_url)

bucket_name = "test"
location = {'LocationConstraint': AWS_REGION}

response = client.create_bucket(Bucket='test', CreateBucketConfiguration=location)

print("Amazon S3 bucket has been created")

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [15]:
import pathlib
import boto3

AWS_REGION = "eu-west-1"
S3_BUCKET_NAME = "hands-on-cloud-demo-bucket"

s3_client = boto3.client("s3", region_name=AWS_REGION, endpoint_url=endpoint_url)

def upload_files(file_name, bucket, object_name=None, args=None):
    if object_name is None:
        object_name = file_name

    s3_client.upload_file(file_name, bucket, object_name, ExtraArgs=args)
    print(f"'{file_name}' has been uploaded to '{S3_BUCKET_NAME}'")

upload_files(f"/home/ndx/Innowise tasks/Innowise_task_8/README.md", S3_BUCKET_NAME)

'/home/ndx/Innowise tasks/Innowise_task_8/README.md' has been uploaded to 'hands-on-cloud-demo-bucket'


In [11]:
s3_resource = boto3.resource("s3", region_name=AWS_REGION, endpoint_url=endpoint_url)

s3_bucket = s3_resource.Bucket(S3_BUCKET_NAME)

print('Listing Amazon S3 Bucket objects/files:')

for obj in s3_bucket.objects.all():
    print(f'-- {obj.key}')

Listing Amazon S3 Bucket objects/files:
-- /home/ndx/Innowise tasks/Innowise_task_8/README.md
-- /home/ndx/Innowise tasks/Innowise_task_8/split_data/split_data_2016-05.csv
-- README.md


In [51]:
s3_resource = boto3.resource("s3", region_name=AWS_REGION, endpoint_url=endpoint_url)

s3_object = s3_resource.Object(S3_BUCKET_NAME, '/home/ndx/Innowise tasks/Innowise_task_8/README.md')

s3_object.download_file('./README.md')

print('S3 object download complete')

S3 object download complete


In [10]:
import io

s3_client = boto3.client("s3", region_name=AWS_REGION, endpoint_url=endpoint_url)
def upload_generated_file_object(bucket, object_name, file_name):

    with open(file_name, 'rb') as f:
        s3_client.upload_fileobj(f, bucket, object_name)
        print(f"Generated has been uploaded to '{bucket}'")

In [40]:
AWS_REGION = "eu-west-1"

client = boto3.client("s3", region_name=AWS_REGION, endpoint_url=endpoint_url)

bucket_name = "hands-on-cloud-demo-bucket"
location = {'LocationConstraint': AWS_REGION}

In [13]:
response = client.create_bucket(Bucket='test', CreateBucketConfiguration=location)

print("Amazon S3 bucket has been created")

Amazon S3 bucket has been created


In [63]:
s3_resource = boto3.resource("s3", region_name=AWS_REGION, endpoint_url=endpoint_url)

def read_file_from_s3(name_of_file):
    s3_object = s3_resource.Object('test', name_of_file)

    with io.BytesIO() as f:
        s3_object.download_fileobj(f)

        f.seek(0)
        return f.read()

read_file_from_s3('README.md')

b'First line.\nSecond line.\n'

In [32]:
upload_generated_file_object('test', 'test.csv', '/home/ndx/Innowise tasks/Innowise_task_8/split_data/test.csv')

Generated has been uploaded to 'test'


In [43]:
import pandas as pd
from io import StringIO

data = read_file_from_s3('test.csv').__str__()[2:-1]
print(data)
data = StringIO(data)
df = pd.read_csv(data, sep=',')
df

departure,return,departure_id,departure_name,return_id,return_name,distance (m),duration (sec.),avg_speed (km/h),departure_latitude,departure_longitude,return_latitude,return_longitude,Air temperature (degC),month\n2016-05-02 09:31:00,2016-05-02 09:32:00,A10,Erottajan aukio,A10,Erottajan aukio,2.0,68.0,0.0017647058823529,60.166890083486,24.9442610720753,60.166890083486,24.9442610720753,14.1,2016-05\n2016-05-02 09:58:00,2016-05-02 09:59:00,A10,Erottajan aukio,A10,Erottajan aukio,2.0,36.0,0.0033333333333333,60.166890083486,24.9442610720753,60.166890083486,24.9442610720753,14.1,2016-05\n


,departure,return,departure_id,departure_name,return_id,return_name,distance (m),duration (sec.),avg_speed (km/h),departure_latitude,...,Erottajan aukio.3,2.0.1,36.0,0.0033333333333333,60.166890083486.2,24.9442610720753.2,60.166890083486.3,24.9442610720753.3,14.1.1,2016-05\n


In [45]:
data = read_file_from_s3('test.csv')
s=str(data,'utf-8')

data = StringIO(s) 

df=pd.read_csv(data)
df

,departure,return,departure_id,departure_name,return_id,return_name,distance (m),duration (sec.),avg_speed (km/h),departure_latitude,departure_longitude,return_latitude,return_longitude,Air temperature (degC),month
0,2016-05-02 09:31:00,2016-05-02 09:32:00,A10,Erottajan aukio,A10,Erottajan aukio,2.0,68.0,0.001765,60.16689,24.944261,60.16689,24.944261,14.1,2016-05
1,2016-05-02 09:58:00,2016-05-02 09:59:00,A10,Erottajan aukio,A10,Erottajan aukio,2.0,36.0,0.003333,60.16689,24.944261,60.16689,24.944261,14.1,2016-05


In [50]:
response = s3_client.list_buckets() 
list(map(lambda item: item['Name'], response['Buckets']))

['hands-on-cloud-demo-bucket', 'test']